Since both Pandas and SQL operate on tabular data, similar operations or queries
can be done using both.

### 1. SELECT

In [19]:
import pandas as pd
# !pip install pandasql
from pandasql import sqldf
import sqlite3

In [20]:
df = pd.read_csv('alert.csv')
#df.drop('Unnamed: 0', axis=1, inplace=True)
#df.drop('rowid', axis=1, inplace=True)

#unning SQL queries in pandas
con = sqlite3.connect('alerts.db')
table = pd.read_sql_query("""SELECT * FROM logs LIMIT 8""",con)
print(table)

   alert_date  alert_metric alert_segment
0  2021-01-05       session       organic
1  2021-01-05       session       organic
2  2021-01-05  transactions        Social
3  2021-01-06      sessions       Organic
4  2021-01-09    bounceRate        Social
5  2021-01-09       session   Paid search
6  2021-01-07  transactions        Social
7  2021-01-06      sessions        Direct


In [21]:
#Running SQL queries in pandas 2
pysql = lambda q: sqldf(q, globals())
df1 = pysql("""SELECT * FROM df LIMIT 8""")
print(df1)

   Unnamed: 0  rowid  alert_date  alert_metric alert_segment
0           0      1  2021-01-05       session       organic
1           1      2  2021-01-05       session       organic
2           2      3  2021-01-05  transactions        Social
3           3      4  2021-01-06      sessions       Organic
4           4      5  2021-01-09    bounceRate        Social
5           5      6  2021-01-09       session   Paid search
6           6      7  2021-01-07  transactions        Social
7           7      8  2021-01-06      sessions        Direct


In [22]:
df.head(8)

,Unnamed: 0,rowid,alert_date,alert_metric,alert_segment
0,0,1,2021-01-05,session,organic
1,1,2,2021-01-05,session,organic
2,2,3,2021-01-05,transactions,Social
3,3,4,2021-01-06,sessions,Organic
4,4,5,2021-01-09,bounceRate,Social
5,5,6,2021-01-09,session,Paid search
6,6,7,2021-01-07,transactions,Social
7,7,8,2021-01-06,sessions,Direct


In [23]:
# SELECT alert_metric, alert_segment ... FROM logs
df.loc[:,['alert_metric','alert_segment']]

,alert_metric,alert_segment
0,session,organic
1,session,organic
2,transactions,Social
3,sessions,Organic
4,bounceRate,Social
5,session,Paid search
6,transactions,Social
7,sessions,Direct
8,bounceRate,Direct
9,session,Paid search


In [24]:
# SELECT DISRINCT alert_metric, alert_segment ... FROM logs
df.loc[:,['alert_metric', 'alert_segment']].drop_duplicates()

,alert_metric,alert_segment
0,session,organic
2,transactions,Social
3,sessions,Organic
4,bounceRate,Social
5,session,Paid search
7,sessions,Direct
8,bounceRate,Direct


<b> SQL’s MIN(), MAX(), COUNT(), AVG(), and SUM() functions are pretty straightforward to
translate to pandas </b>:

In [30]:
#SELECT AVG(col) FROM table
#df.loc[:,['alert_date']].max()
#df.loc[:,['alert_date']].min()
#df.loc[:,['alert_date']].sum()

### 2. ORDER

In [31]:
df

,Unnamed: 0,rowid,alert_date,alert_metric,alert_segment
0,0,1,2021-01-05,session,organic
1,1,2,2021-01-05,session,organic
2,2,3,2021-01-05,transactions,Social
3,3,4,2021-01-06,sessions,Organic
4,4,5,2021-01-09,bounceRate,Social
5,5,6,2021-01-09,session,Paid search
6,6,7,2021-01-07,transactions,Social
7,7,8,2021-01-06,sessions,Direct
8,8,9,2021-01-07,bounceRate,Direct
9,9,10,2021-01-09,session,Paid search


In [32]:
#SELECT col FROM table WHERE col=... BETWEEN ...
df.loc[(df['alert_date'] > '2021-01-05') & (df['alert_date'] < '2021-01-07')]

,Unnamed: 0,rowid,alert_date,alert_metric,alert_segment
3,3,4,2021-01-06,sessions,Organic
7,7,8,2021-01-06,sessions,Direct


In [33]:
# SELECT col1, col2, ... FROM table ORDER BY col1, col2 ASC|DESC
df.loc[(df['alert_date'] > '2021-01-05')].sort_values(by=['alert_segment'], ascending=True)

,Unnamed: 0,rowid,alert_date,alert_metric,alert_segment
7,7,8,2021-01-06,sessions,Direct
8,8,9,2021-01-07,bounceRate,Direct
3,3,4,2021-01-06,sessions,Organic
5,5,6,2021-01-09,session,Paid search
9,9,10,2021-01-09,session,Paid search
4,4,5,2021-01-09,bounceRate,Social
6,6,7,2021-01-07,transactions,Social


### 3. GROUP BY

In [34]:
df.loc[:,:].groupby(['alert_date']).count()

,Unnamed: 0,rowid,alert_metric,alert_segment
alert_date,,,,
2021-01-05,3,3,3,3
2021-01-06,2,2,2,2
2021-01-07,2,2,2,2
2021-01-09,3,3,3,3


### 4. INSERT

In [35]:
# In pandas we can use the .append() method to append a new row at the end of an existing dataframe.
# We will use ignore_index=True in order to continue indexing from the last row in the old data frame.


# INSERT INTO table (column1, column2, ...) VALUES (value1, value2, ...)

df = df.append({'alert_date':'2021-10-06',
               'alert_metric':'transaction',
               'alert_segment':'organic'}, ignore_index=True)
df

/tmp/ipykernel_52308/4110397432.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'alert_date':'2021-10-06',


,Unnamed: 0,rowid,alert_date,alert_metric,alert_segment
0,0.0,1.0,2021-01-05,session,organic
1,1.0,2.0,2021-01-05,session,organic
2,2.0,3.0,2021-01-05,transactions,Social
3,3.0,4.0,2021-01-06,sessions,Organic
4,4.0,5.0,2021-01-09,bounceRate,Social
5,5.0,6.0,2021-01-09,session,Paid search
6,6.0,7.0,2021-01-07,transactions,Social
7,7.0,8.0,2021-01-06,sessions,Direct
8,8.0,9.0,2021-01-07,bounceRate,Direct
9,9.0,10.0,2021-01-09,session,Paid search


### 5. DELETE

In [36]:
# DELETE FROM table WHERE condition

import numpy as np

df = df.drop(np.where(df['alert_segment']=='Social')[0])
df.reset_index(drop=True, inplace=True)

In [37]:
df

,Unnamed: 0,rowid,alert_date,alert_metric,alert_segment
0,0.0,1.0,2021-01-05,session,organic
1,1.0,2.0,2021-01-05,session,organic
2,3.0,4.0,2021-01-06,sessions,Organic
3,5.0,6.0,2021-01-09,session,Paid search
4,7.0,8.0,2021-01-06,sessions,Direct
5,8.0,9.0,2021-01-07,bounceRate,Direct
6,9.0,10.0,2021-01-09,session,Paid search
7,NaN,NaN,2021-10-06,transaction,organic


### 6. ALTER
This SQL statement adds new columns

In [38]:
# ALTER TABLE table ADD column
sensitivity_level = ['high', 'low', 'low', 'moderate', 'moderate', 'low', 'high', 'low']

df['sensitivity_level'] = sensitivity_level


In [39]:
sensitivity_rate = [4.7, 0.6, 1.4, 3.0, 2.7, 1.2, 4.7, 1.7]
df['sensitivity_rate'] = sensitivity_rate

In [40]:
df

,Unnamed: 0,rowid,alert_date,alert_metric,alert_segment,sensitivity_level,sensitivity_rate
0,0.0,1.0,2021-01-05,session,organic,high,4.7
1,1.0,2.0,2021-01-05,session,organic,low,0.6
2,3.0,4.0,2021-01-06,sessions,Organic,low,1.4
3,5.0,6.0,2021-01-09,session,Paid search,moderate,3.0
4,7.0,8.0,2021-01-06,sessions,Direct,moderate,2.7
5,8.0,9.0,2021-01-07,bounceRate,Direct,low,1.2
6,9.0,10.0,2021-01-09,session,Paid search,high,4.7
7,NaN,NaN,2021-10-06,transaction,organic,low,1.7


In [41]:
!